In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [7]:
train = pd.read_csv('F:\\JinnanPreliminary\\jinnan_round1_train_20181227.csv', encoding = 'gb18030')
#
test  = pd.read_csv('F:\\JinnanPreliminary\\jinnan_round1_testB_20190121.csv', encoding = 'gb18030')
sample_id=test["样本id"]
target_col = "收率"

# 删除异常值
print(train[train['收率'] < 0.87])

train = train[train['收率'] > 0.87]
train.loc[train['B14'] == 40, 'B14'] = 400
train = train[train['B14']>=400]

# 合并数据集, 顺便处理异常数据
target = train['收率']
train.loc[train['A25'] == '1900/3/10 0:00', 'A25'] = train['A25'].value_counts().values[0]
train['A25'] = train['A25'].astype(int)
train.loc[train['B14'] == 40, 'B14'] = 400
# test.loc[test['B14'] == 385, 'B14'] = 385

test_select = {}
for v in [280, 385, 390, 785]:
    print(v)
    print(test[test['B14'] == v]['样本id'])
    test_select[v] = test[test['B14'] == v]['样本id'].index
    print(test[test['B14'] == v]['样本id'].index)
    print(test_select[v])

del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

def timeTranSecond(t):
    try:
        t, m, s = t.split(":")
    except:
        if t == '1900/1/9 7:00':
            return 7 * 3600 / 3600
        elif t == '1900/1/1 2:30':
            return (2 * 3600 + 30 * 60) / 3600
        elif t == -1:
            return -1
        else:
            return 0

    try:
        tm = (int(t) * 3600 + int(m) * 60 + int(s)) / 3600
    except:
        return (30 * 60) / 3600

    return tm


for f in ['A5', 'A7', 'A9', 'A11', 'A14', 'A16', 'A24', 'A26', 'B5', 'B7']:
    try:
        data[f] = data[f].apply(timeTranSecond)
    except:
        print(f, '应该在前面被删除了！')


def getDuration(se):
    try:
        sh, sm, eh, em = re.findall(r"\d+\.?\d*", se)
    except:
        if se == -1:
            return -1

    try:
        if int(sh) > int(eh):
            tm = (int(eh) * 3600 + int(em) * 60 - int(sm) * 60 - int(sh) * 3600) / 3600 + 24
        else:
            tm = (int(eh) * 3600 + int(em) * 60 - int(sm) * 60 - int(sh) * 3600) / 3600
    except:
        if se == '19:-20:05':
            return 1
        elif se == '15:00-1600':
            return 1
    return tm


for f in ['A20', 'A28', 'B4', 'B9', 'B10', 'B11']:
    data[f] = data.apply(lambda df: getDuration(df[f]), axis=1)

data['样本id'] = data['样本id'].apply(lambda x: x.split('_')[1])
data['样本id'] = data['样本id'].astype(int)
data.drop(['样本id'],axis=1,inplace=True)
df_data=data.copy()

categorical_columns = [f for f in data.columns if f not in ['样本id']]
numerical_columns = [f for f in data.columns if f not in categorical_columns]
#label encoder
for f in categorical_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
# 基本数据处理完毕,

             样本id   A1     A2     A3   A4        A5    A6   A7  A8        A9  \
120    sample_842  300  125.0    NaN  980  14:30:00  24.0  NaN NaN  16:00:00   
138   sample_1001  300    NaN  405.0  700   1:00:00  21.0  NaN NaN   2:30:00   
248   sample_1040  200    NaN  270.0  470   9:30:00  21.0  NaN NaN  11:00:00   
322    sample_996  300    NaN  405.0  700   4:00:00  30.0  NaN NaN   5:00:00   
447    sample_994  300    NaN  405.0  700  10:40:00  28.0  NaN NaN  11:50:00   
484    sample_752  300    NaN  405.0  700  20:00:00  28.0  NaN NaN  22:00:00   
687    sample_748  300    NaN  405.0  700  15:00:00  30.0  NaN NaN  17:00:00   
762    sample_849  300  125.0    NaN  980   5:30:00  21.0  NaN NaN   7:00:00   
869    sample_293  300  125.0    NaN  980  15:00:00  21.0  NaN NaN  16:00:00   
956    sample_450  300    NaN  405.0  700  23:00:00  21.0  NaN NaN   0:30:00   
1012   sample_751  300    NaN  405.0  700   8:30:00  28.0  NaN NaN  10:30:00   
1195   sample_750  300    NaN  405.0  70

In [8]:
#multiply
col_list=df_data.columns
for col,index in zip(col_list,range(df_data.shape[1])):
    for _col in col_list[index:]:
        data[col+"multiply"+_col]=df_data[col]*df_data[_col]
        print(col,'*',_col)
print("-------------------------------------------",data.shape,"-------------------------------------------")
dispersion
mean_array=df_data.mean(axis=0)
dispersion_mean_list=[]
for col in df_data.columns:
    dispersion_mean_list.append(col+"_dispersion")
data[dispersion_mean_list]=df_data-mean_array
print("-------------------------------------------",data.shape,"-------------------------------------------")     
#add_minus
# numerical_col_list=['A6', 'A8', 'A10', 'A12', 'A15', 'A17', 'A19', 'A21', 'A22', 'A25', 'A27', 'B1', 'B6', 'B8', 'B12', 'B14']

# for col,index in zip(numerical_col_list,range(len(numerical_col_list))):
#     for _col in numerical_col_list[index:]:
#         data[col+"add"+_col]=df_data[col]+df_data[_col]
#         data[col+"minus"+_col]=df_data[col]-df_data[_col]
#         print(col,'+',_col)
#         print(col,'-',_col)
# print("-------------------------------------------",data.shape,"-------------------------------------------")
#unreasonable feature
data['b14/a1_a3_a4_a19_b1_b12'] = df_data['B14']/(df_data['A1']+df_data['A3']+df_data['A4']+df_data['A19']+df_data['B1']+df_data['B12'])
print("-------------------------------------------",data.shape,"-------------------------------------------")

A1 * A1
A1 * A2
A1 * A3
A1 * A4
A1 * A5
A1 * A6
A1 * A7
A1 * A8
A1 * A9
A1 * A10
A1 * A11
A1 * A12
A1 * A13
A1 * A14
A1 * A15
A1 * A16
A1 * A17
A1 * A18
A1 * A19
A1 * A20
A1 * A21
A1 * A22
A1 * A23
A1 * A24
A1 * A25
A1 * A26
A1 * A27
A1 * A28
A1 * B1
A1 * B2
A1 * B3
A1 * B4
A1 * B5
A1 * B6
A1 * B7
A1 * B8
A1 * B9
A1 * B10
A1 * B11
A1 * B12
A1 * B13
A1 * B14
A2 * A2
A2 * A3
A2 * A4
A2 * A5
A2 * A6
A2 * A7
A2 * A8
A2 * A9
A2 * A10
A2 * A11
A2 * A12
A2 * A13
A2 * A14
A2 * A15
A2 * A16
A2 * A17
A2 * A18
A2 * A19
A2 * A20
A2 * A21
A2 * A22
A2 * A23
A2 * A24
A2 * A25
A2 * A26
A2 * A27
A2 * A28
A2 * B1
A2 * B2
A2 * B3
A2 * B4
A2 * B5
A2 * B6
A2 * B7
A2 * B8
A2 * B9
A2 * B10
A2 * B11
A2 * B12
A2 * B13
A2 * B14
A3 * A3
A3 * A4
A3 * A5
A3 * A6
A3 * A7
A3 * A8
A3 * A9
A3 * A10
A3 * A11
A3 * A12
A3 * A13
A3 * A14
A3 * A15
A3 * A16
A3 * A17
A3 * A18
A3 * A19
A3 * A20
A3 * A21
A3 * A22
A3 * A23
A3 * A24
A3 * A25
A3 * A26
A3 * A27
A3 * A28
A3 * B1
A3 * B2
A3 * B3
A3 * B4
A3 * B5
A3 * B6
A3 * B7
A3 * 

NameError: name 'dispersion' is not defined

In [9]:
# 特征提取一段结束 开始拼接数据
train = data[:train.shape[0]]
test  = data[train.shape[0]:]
train['target'] = list(target)
print(train.shape)
print(test.shape)

(1330, 946)
(149, 945)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
train.columns

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       ...
       'B12multiplyB14', 'B13multiplyB13', 'B13multiplyB14', 'B14multiplyB14',
       'target', 'intTarget_0', 'intTarget_1', 'intTarget_2', 'intTarget_3',
       'intTarget_4'],
      dtype='object', length=951)

In [11]:
# li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
li = ['intTarget_0','intTarget_1','intTarget_2','intTarget_3','intTarget_4']
mean_columns = []
for f1 in categorical_columns:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            order_label = train.groupby([f1])[f2].mean()
            print("order_label",order_label)
            train[col_name] = train['B14'].map(order_label)
            print("B14",train['B14'])
            print("train",col_name,train[col_name])
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)
train.drop(li, axis=1, inplace=True)

order_label A5
0     0.400000
1     0.054475
2     0.333333
3     0.102410
4     0.173913
5     0.111111
6     0.063197
7     0.375000
8     0.190476
9     0.151163
10    0.166667
11    0.400000
12    0.000000
13    0.500000
14    0.054054
15    0.028571
16    0.250000
17    0.210526
18    0.428571
19    0.428571
20    1.000000
21    0.166667
22    0.357143
23    0.272727
24    0.200000
25    0.333333
26    0.000000
27    0.250000
28    0.333333
29    0.333333
        ...   
33    0.666667
34    0.222222
35    0.333333
36    0.600000
37    0.238095
38    1.000000
39    0.181818
40    0.166667
41    0.500000
42    0.000000
43    0.090909
44    0.200000
45    0.000000
46    0.000000
47    0.000000
48    0.000000
49    0.285714
50    1.000000
51    1.000000
52    0.500000
53    1.000000
54    0.500000
55    0.000000
56    0.500000
57    1.000000
58    0.000000
59    0.000000
60    0.000000
61    0.500000
62    0.000000
Name: intTarget_0, Length: 63, dtype: float64
B14 0       0
1       0


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



order_label A5
0     0.000000
1     0.182879
2     0.000000
3     0.150602
4     0.130435
5     0.055556
6     0.063197
7     0.000000
8     0.000000
9     0.069767
10    0.000000
11    0.000000
12    0.250000
13    0.000000
14    0.243243
15    0.000000
16    0.000000
17    0.052632
18    0.047619
19    0.000000
20    0.000000
21    0.000000
22    0.000000
23    0.090909
24    0.000000
25    0.000000
26    0.000000
27    0.250000
28    0.000000
29    0.000000
        ...   
33    0.000000
34    0.000000
35    0.066667
36    0.000000
37    0.047619
38    0.000000
39    0.090909
40    0.000000
41    0.000000
42    0.250000
43    0.090909
44    0.200000
45    0.000000
46    0.000000
47    0.000000
48    0.000000
49    0.000000
50    0.000000
51    0.000000
52    0.500000
53    0.000000
54    0.250000
55    0.000000
56    0.000000
57    0.000000
58    0.000000
59    0.000000
60    0.000000
61    0.000000
62    0.000000
Name: intTarget_3, Length: 63, dtype: float64
B14 0       0
1       0

train B14_to_A6_intTarget_2_mean 0       0.259259
1       0.259259
2       0.259259
3       0.259259
4       0.480730
5       0.480730
6       0.259259
7       0.259259
8       0.480730
9       0.259259
10      0.259259
11      0.259259
12      0.259259
13      0.259259
14      0.259259
15      0.259259
16      0.480730
17      0.259259
18      0.259259
19      0.259259
20      0.333333
21      0.333333
22      0.333333
23      0.480730
24      0.480730
25      0.480730
26      0.333333
27      0.480730
28      0.259259
29      0.259259
          ...   
1300    0.333333
1301    0.259259
1302    0.259259
1303    0.259259
1304    0.480730
1305    0.259259
1306    0.259259
1307    0.259259
1308    0.333333
1309    0.259259
1310    0.259259
1311    0.259259
1312    0.259259
1313    0.259259
1314    0.259259
1315    0.480730
1316    0.259259
1317    0.259259
1318    0.480730
1319    0.250000
1320    0.480730
1321    0.259259
1322    0.333333
1323    0.259259
1324    0.250000
1325    0.33333

order_label A9
0     0.200000
1     0.516393
2     0.375000
3     0.348993
4     0.111111
5     0.043478
6     0.599206
7     0.320000
8     0.312500
9     0.644444
10    0.250000
11    0.100000
12    0.222222
13    0.333333
14    0.285714
15    0.375000
16    0.384615
17    0.250000
18    0.333333
19    0.000000
20    0.200000
21    0.000000
22    0.250000
23    0.000000
24    1.000000
25    0.176471
26    0.428571
27    0.400000
28    0.000000
29    0.153846
        ...   
58    0.090909
59    0.000000
60    1.000000
61    0.000000
62    0.000000
63    0.000000
64    0.000000
65    0.090909
66    0.000000
67    0.000000
68    0.000000
69    1.000000
70    0.000000
71    0.000000
72    0.000000
73    0.166667
74    0.000000
75    0.000000
76    0.000000
77    0.000000
78    0.000000
79    0.000000
80    0.000000
81    0.000000
82    0.000000
83    0.000000
84    1.000000
85    1.000000
86    0.000000
87    0.000000
Name: intTarget_2, Length: 88, dtype: float64
B14 0       0
1       0


...

order_label B8
0     0.279048
1     0.384615
2     0.500000
3     0.472441
4     0.200000
5     0.666667
6     0.285714
7     0.200000
8     0.500000
9     0.500000
10    0.600000
11    0.250000
12    0.333333
13    0.500000
14    0.250000
15    1.000000
16    0.571429
17    0.714286
18    0.000000
19    0.200000
20    0.000000
21    0.000000
22    0.000000
23    0.000000
24    0.000000
Name: intTarget_1, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
13

Name: B14_to_B9_intTarget_1_mean, Length: 1330, dtype: float64
order_label B9
0     0.329897
1     0.666667
2     0.725806
3     0.211538
4     0.058824
5     0.739130
6     0.166667
7     0.000000
8     0.636364
9     0.666667
10    0.450000
11    0.000000
12    1.000000
13    0.000000
14    0.000000
Name: intTarget_2, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_t

order_label B10
0     0.329386
1     0.716814
2     0.600000
3     0.066667
4     0.250000
5     0.166667
6     0.666667
7     0.304348
8     1.000000
9     0.428571
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    1.000000
16    0.000000
Name: intTarget_2, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B10_intTarget_2_mean 0       0

order_label B11
0    0.359694
1    0.425243
2    0.565217
3    0.000000
4    0.500000
5    0.000000
6    1.000000
7    1.000000
8    0.000000
Name: intTarget_2, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B11_intTarget_2_mean 0       0.359694
1       0.359694
2       0.359694
3       0.359694
4       0.425243
5       0.425243
6       0.359694
7       0.359694
8 

order_label B12
0    0.022346
1    0.090674
2    0.000000
3    0.000000
Name: intTarget_4, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B12_intTarget_4_mean 0       0.022346
1       0.022346
2       0.022346
3       0.022346
4       0.090674
5       0.090674
6       0.022346
7       0.022346
8       0.090674
9       0.022346
10      0.022346
11      0.022346
12  

In [12]:
label_array=train['target']
train.drop(['target'],axis=1,inplace=True)
train_array=train.values
test_array=test.values
print(train.shape,test.shape)
print(train_array.shape,test_array.shape)

(1330, 1075) (149, 1075)
(1330, 1075) (149, 1075)


In [13]:
from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [21]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(train_array.shape[0])
predictions_lgb = np.zeros(test_array.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train_array[trn_idx], label_array[trn_idx])
    val_data = lgb.Dataset(train_array[val_idx], label_array[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(train_array[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test_array, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, label_array)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000177944	valid_1's l2: 0.00023413
[400]	training's l2: 0.000132756	valid_1's l2: 0.000202045
[600]	training's l2: 0.000113565	valid_1's l2: 0.000198675
[800]	training's l2: 0.00010345	valid_1's l2: 0.000198491
Early stopping, best iteration is:
[778]	training's l2: 0.000104286	valid_1's l2: 0.000198288
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000181694	valid_1's l2: 0.000196573
[400]	training's l2: 0.000135836	valid_1's l2: 0.000173338
[600]	training's l2: 0.000115421	valid_1's l2: 0.000168295
[800]	training's l2: 0.000104488	valid_1's l2: 0.000167247
Early stopping, best iteration is:
[850]	training's l2: 0.00010255	valid_1's l2: 0.000167002
fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000186787	valid_1's l2: 0.000194461
[400]	training's l2: 0.00014128	valid_1's l2: 0.000169954
[600]	trai

In [22]:
# df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_lgb})
# df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
# df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_unreason_lgb_1_19.csv",header=None,index=False)
print("mean_lgb:",np.mean(predictions_lgb))
print("var_lgb:",np.var(predictions_lgb))

mean_lgb: 0.9259743195306993
var_lgb: 0.0006154048475984998


In [16]:
xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8,
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(train_array.shape[0])
predictions_xgb = np.zeros(test_array.shape[0])
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = xgb.DMatrix(train_array[trn_idx], label_array[trn_idx])
    val_data = xgb.DMatrix(train_array[val_idx], label_array[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                    verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(train_array[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, label_array)))

fold n°1
[0]	train-rmse:0.424077	valid_data-rmse:0.424312
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.257689	valid_data-rmse:0.258012
[200]	train-rmse:0.156986	valid_data-rmse:0.157207
[300]	train-rmse:0.096066	valid_data-rmse:0.096129
[400]	train-rmse:0.059351	valid_data-rmse:0.059404
[500]	train-rmse:0.03742	valid_data-rmse:0.037832
[600]	train-rmse:0.024441	valid_data-rmse:0.025662
[700]	train-rmse:0.01684	valid_data-rmse:0.019304
[800]	train-rmse:0.012469	valid_data-rmse:0.016303
[900]	train-rmse:0.010029	valid_data-rmse:0.015034
[1000]	train-rmse:0.008655	valid_data-rmse:0.014539
[1100]	train-rmse:0.007864	valid_data-rmse:0.014397
[1200]	train-rmse:0.007391	valid_data-rmse:0.014405
[1300]	train-rmse:0.007076	valid_data-rmse:0.014444
Stopping. Best iteration:
[1142]	train-rmse:0.00764	valid_data-rmse:0.014388

fold n°2
[0]	train-rmse:0.424443	valid_data

In [18]:
# df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_xgb})
# df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
# df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_idfea_binning_dispersion_xgb_1_11.csv",header=None,index=False)
print("mean_xgb:",np.mean(predictions_xgb))
print("var_xgb:",np.var(predictions_xgb))

mean_xgb: 0.9239857543834904
var_xgb: 0.0006182350439788258


In [19]:
from sklearn.linear_model import BayesianRidge
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,label_array)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], label_array[trn_idx]
    val_data, val_y = train_stack[val_idx], label_array[val_idx]
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(label_array, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


0.0001756834879469391

In [20]:
# df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions})
# df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
# df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_idfea_binning_dispersion_stacking_1_11.csv",header=None,index=False)

print("mean:",np.mean(predictions))
print("var:",np.var(predictions))

mean: 0.9257415432332686
var: 0.0006136764928118953
